In [6]:
import google.generativeai as genai
import ast
import os
import json
import PIL.Image
from browser import BrowserAutomation
from selenium.webdriver.common.by import By
from constants import  *
from dotenv import load_dotenv

load_dotenv()

GEMINI_API_KEY = os.environ.get("GOOGLE_GENERATIVE_AI_API_KEY")

url = "https://www.amazon.com/"
file = "website.html"
prompt = "I want to see the latest deals"

In [7]:
def generate_content_with_cycling_keys(prompt, system_prompt, image=None):
    api_key = GEMINI_API_KEY

    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(
        "gemini-1.5-pro-latest",
        generation_config=genai.GenerationConfig(
            max_output_tokens=8000,
            temperature=0,
        ),
        system_instruction=system_prompt,
    )

    if image is None:
        response = model.generate_content(prompt, request_options={"timeout": 1000})
    else:
        response = model.generate_content(
            [prompt, image], request_options={"timeout": 1000}
        )
    return response.text

In [14]:
browser = BrowserAutomation()
browser.open_browser()

html_string = browser.scrape()

screenshot = browser.take_screenshot("website.png")
img = PIL.Image.open('website.png')
url = browser.current_url
prompt = "I would like to order a pizza for delivery."

user_prompt = f"""
current_url: {url}
current_page: {html_string}
current_screenshot is attached
Output selectors for relevant elements (divs, inputs, and images) that are relevant to the user's request.

user: {prompt}
"""
    
response = generate_content_with_cycling_keys(user_prompt, system_prompt_interpret, img)
if "```json" in response:
    response = response.split("```json")[1].split("```")[0]
obj = json.loads(response)
obj

TypeError: BrowserAutomation.take_screenshot() takes 1 positional argument but 2 were given